

**1. importing modules**

Always run steps 1 and 2 first, then you can run the others separately.


In [1]:
# importing modules
import pandas as pd
import os
from datetime import date
from datetime import datetime
import time
import openpyxl
from openpyxl.worksheet.datavalidation import DataValidation
import gc
from openpyxl import load_workbook

**2. merging all the tabs into one file**

first, download the list of users from EMR (**logging in --> menu --> setup --> users --> there is a button left next to "create users" called "export", click on "export" --> wait for the file to load and export to location of choice in computer**), upload this file into co-laboratories, change the pathway to the file on "excel_file", run the script, then wait for an output file called "merged_data" to come out.

This code does several things:

1. It reads data from an Excel file with multiple sheets into separate dataframes.
2. It merges these dataframes together based on a common column ('Username' in this case).
3. It then writes the merged dataframe into a new Excel file.

So, basically, it's combining information from different sheets in an Excel file into one big sheet and saving it as a new file.

In [ ]:
# Read Excel sheets into dataframes
excel_file = "/content/Colorado_Users_03_29_2024_12_02.xlsx"
sheet_names = ["Profile", "Notification Addresses", "Roles", "Resource Associations"]  # Replace with your sheet names
dfs = {}  # Dictionary to store dataframes
for sheet_name in sheet_names:
    dfs[sheet_name] = pd.read_excel(excel_file, sheet_name=sheet_name)

# Merge dataframes based on the 'username' column
merged_df = None
for df in dfs.values():
    if merged_df is None:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, on='Username', how='left')

# Write merged dataframe to Excel
with pd.ExcelWriter("merged_data.xlsx") as writer:
    merged_df.to_excel(writer, index=False, sheet_name="MergedSheet")


**3. creating data for the "role audit" tab**

So here, we will use the output "merged_data" file to create the data for the "Role audit" tab on this spreadsheet "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0".

This code does several things with an Excel file:

1. It reads data from an Excel file into a DataFrame (think of it as a table).
2. It removes duplicate rows based on specific columns (like removing repeated entries).
3. It loads the same Excel file again.
4. It creates a list of roles.
5. It adds new columns to the DataFrame for each role, marking them with checkmarks if a user has that role. If the user has more than one roles, it will not be grouped into one user (so that user may have duplicate username if they have multiple roles)
6. It writes this updated DataFrame to a new Excel file.
7. It adjusts the width of columns in the Excel file.
8. It saves the updated Excel file.
9. It reads the updated Excel file into a DataFrame again.
10. It selects specific columns from the DataFrame.
11. It saves the selected columns to another Excel file.
12. Finally, it prints a message confirming where the output file is saved.

Copy and paste the results after doing that to the spreadsheet here "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0"

In [3]:
# Read the Excel file into a DataFrame
excel_file = "/content/merged_data.xlsx"
df = pd.read_excel(excel_file)

# Remove duplicates based on the "Username" column
df.drop_duplicates(subset=["Username", "Role"], inplace=True)


In [6]:
# Load the Excel workbook
workbook = openpyxl.load_workbook('/content/merged_data.xlsx')

# Assuming 'df' is your original DataFrame
data = pd.DataFrame(df)

# Define a list of roles
roles = ['Behavioral Health - State Admin', 'Behavioral Health', 'Coroner', 'Dispatcher',
         'ED Admin / Hospital Admin', 'Emergency Dept.', 'Emergency Managers', 'ESF-8',
         'Fire / EMS', 'Health Facilities - State Admin', 'Health Facilities and Clinics',
         'Healthcare Coalition', 'Long Term Care', 'MRC - Medical Reserve Corp',
         'MSPDC - Event creation only', 'OEPR', 'Pharmacy', 'ReadOnly',
         'Regional Staff / HCC Cord', 'State System Administrator']

# Add columns for each role with checkmarks
for role in roles:
    data[role] = data['Role'].apply(lambda x: '☑' if x == role else '')

# Create an Excel writer object
excel_writer = pd.ExcelWriter('user_roles.xlsx', engine='openpyxl')

# Write the DataFrame to the Excel file
data.to_excel(excel_writer, sheet_name='User Roles', index=False)

# Save the Excel file
excel_writer.close()


In [7]:
# Read the Excel file into a DataFrame
excel_file = "/content/user_roles.xlsx"
df = pd.read_excel(excel_file)

# Selecting only the desired columns
selected_columns = ['Username', 'Login Email', 'Last login', 'Behavioral Health - State Admin', 'Behavioral Health', 'Coroner', 'Dispatcher',
         'ED Admin / Hospital Admin', 'Emergency Dept.', 'Emergency Managers', 'ESF-8',
         'Fire / EMS', 'Health Facilities - State Admin', 'Health Facilities and Clinics',
         'Healthcare Coalition', 'Long Term Care', 'MRC - Medical Reserve Corp',
         'MSPDC - Event creation only', 'OEPR', 'Pharmacy', 'ReadOnly',
         'Regional Staff / HCC Cord', 'State System Administrator']
df_selected = df[selected_columns]

# Saving the output to an Excel file
output_file = "role_audit.xlsx"
df_selected.to_excel(output_file, index=False)

print(f"Output saved to {output_file}")

Output saved to role_audit.xlsx


**4. creating data for the "resource" tab**

So here, we will use the output "merged_data" file to create the data for the "resource" tab on this spreadsheet "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0".  

This script performs the following tasks:

1. It reads an Excel file named "merged_data.xlsx" into a pandas DataFrame.
2. It groups the data by different combinations of columns ('Resource (Name)', 'Associated', 'Update', 'Report', 'Administer User', 'Can Assign (Yes/No)'), and counts the number of unique users (by counting unique usernames).
3. For each group, it drops duplicate entries, keeping only the last occurrence.
4. It merges these grouped dataframes into a single dataframe based on the 'Resource (Name)' column, using a left join strategy.
5. Finally, it saves the merged dataframe into a new Excel file named "resource.xlsx" without including the index.

This script is analyzing user roles or permissions associated with different resources, and it consolidates this information into a single Excel file for further analysis or reporting.

In the Excel spreadsheet, we specifically require counts for individuals who responded with 'yes.' Therefore, upon downloading the sheet, please filter out all instances of 'associated,' 'update,' 'report,' 'delegated admin,' and 'assign' where the response is 'No,' and input '0' for those entries. Copy and paste the results after doing that to the spreadsheet here "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0"

In [12]:
# Read the Excel file into a DataFrame
excel_file = "/content/merged_data.xlsx"
df = pd.read_excel(excel_file)

In [13]:
# Group by the "Resource" column and count the number of unique users
role_counts_1 = df.groupby(['Resource (Name)', 'Associated'])['Username'].nunique().reset_index()
role_counts_1 = role_counts_1.drop_duplicates(subset = 'Resource (Name)', keep = 'last')
role_counts_2 = df.groupby(['Resource (Name)', 'Update'])['Username'].nunique().reset_index()
role_counts_2 = role_counts_2.drop_duplicates(subset = 'Resource (Name)', keep = 'last')
role_counts_3 = df.groupby(['Resource (Name)', 'Report'])['Username'].nunique().reset_index()
role_counts_3 = role_counts_3.drop_duplicates(subset = 'Resource (Name)', keep = 'last')
role_counts_4 = df.groupby(['Resource (Name)', 'Delegated Admin'])['Username'].nunique().reset_index()
role_counts_4 = role_counts_4.drop_duplicates(subset = 'Resource (Name)', keep = 'last')
role_counts_5 = df.groupby(['Resource (Name)', 'Can Assign (Yes/No)'])['Username'].nunique().reset_index()
role_counts_5 = role_counts_5.drop_duplicates(subset = 'Resource (Name)', keep = 'last')

In [21]:
# Merging multiple dataframes using a loop
dfs = [role_counts_1, role_counts_2, role_counts_3, role_counts_4, role_counts_5]
merged_multiple = role_counts_1  # Initialize with the first dataframe

for i, df in enumerate(dfs[1:], start=2):  # Start from index 2 to use suffixes
    merged_multiple = pd.merge(merged_multiple, df, on='Resource (Name)', how='left', suffixes=('', f'_{i}'))

# Save the merged data to a new Excel file
merged_multiple.to_excel('resource_tab.xlsx', index=False)


**5. creating data for the "user" tab**

So here, we will use the output "merged_data" file to create the data for the "user" tab on this spreadsheet "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0".

This code does several things:

1. It reads data from an Excel file named "merged_data.xlsx" into a DataFrame.
2. It groups the data by different categories (like roles), counts the number of unique users for each category, and stores these counts in separate DataFrames.
3. It merges these separate DataFrames into one based on a common column, "Username".
4. It drops some unnecessary columns from the merged DataFrame.
5. Finally, it saves the merged and modified data into a new Excel file named "user_tab.xlsx".

In the Excel spreadsheet, we specifically require counts for individuals who responded with 'yes.' Therefore, upon downloading the sheet, please filter out all instances of 'associated,' 'update,' 'report,' 'delegated admin,' and 'assign' where the response is 'No,' and input '0' for those entries. Copy and paste the results after doing that to the spreadsheet here "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0"

In [22]:
# Read the Excel file into a DataFrame
excel_file = "/content/merged_data.xlsx"
df = pd.read_excel(excel_file)

In [23]:
# Group by the "roles" column and count the number of unique users
role_counts_1 = df.groupby(['Username', 'Associated'])['Resource (Name)'].nunique().reset_index()
role_counts_1 = role_counts_1.drop_duplicates(subset = 'Username', keep = 'last')
role_counts_2 = df.groupby(['Username', 'Update'])['Resource (Name)'].nunique().reset_index()
role_counts_2 = role_counts_2.drop_duplicates(subset = 'Username', keep = 'last')
role_counts_3 = df.groupby(['Username', 'Report'])['Resource (Name)'].nunique().reset_index()
role_counts_3 = role_counts_3.drop_duplicates(subset = 'Username', keep = 'last')
role_counts_4 = df.groupby(['Username', 'Delegated Admin'])['Resource (Name)'].nunique().reset_index()
role_counts_4 = role_counts_4.drop_duplicates(subset = 'Username', keep = 'last')
role_counts_5 = df.groupby(['Username', 'Can Assign (Yes/No)'])['Resource (Name)'].nunique().reset_index()
role_counts_5 = role_counts_5.drop_duplicates(subset = 'Username', keep = 'last')

In [26]:
# Merging multiple dataframes using a loop
dfs = [role_counts_1, role_counts_2, role_counts_3, role_counts_4, role_counts_5]
merged_multiple = role_counts_1  # Initialize with the first dataframe

for i, df in enumerate(dfs[1:], start=2):  # Start from index 2 to use suffixes
    merged_multiple = pd.merge(merged_multiple, df, on='Username', how='left', suffixes=('', f'_{i}'))

# Save the merged data to a new Excel file
merged_multiple.to_excel('user_tab.xlsx', index=False)



**6. creating data for the "users by year" tab**

So here, we will use the output "merged_data" file to create the data for the "users by year" on this spreadsheet "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0".

Here's what's happening in simpler terms:

1. The code reads data from an Excel file named "merged_data.xlsx" and stores it in a pandas DataFrame (a table-like structure for handling data in Python).

2. It removes any duplicate rows in the DataFrame based on the "Username" column. This ensures each user appears only once in the data.

3. It saves the cleaned data (without duplicates) into a new Excel file named "result.xlsx".

4. It reads the cleaned data from "result.xlsx" into another DataFrame called "data".

5. It converts the "Last login" column in the DataFrame to datetime format, which is a special type for handling dates and times in Python.

6. It extracts the year from the "Last login" column and creates a new column named "login_year" to store these years.

7. It counts the number of users for each year based on their last login and stores this count in a new DataFrame called "user_counts".

8. Finally, it saves the user counts for each year into a new Excel file named "users_by_year.xlsx".

In short, this code reads data from an Excel file, cleans it by removing duplicates, analyzes login information by extracting years, and saves the results into new Excel files.

Copy and paste results on this "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0"

In [27]:
# Read the Excel file into a DataFrame
excel_file = "/content/merged_data.xlsx"
df = pd.read_excel(excel_file)

# Remove duplicates based on the "Username" column
df.drop_duplicates(subset=["Username"], inplace=True)

# Save the merged data to a new Excel file
df.to_excel('result.xlsx', index=False)

In [28]:
# Read your data into a pandas DataFrame
data = pd.read_excel("/content/result.xlsx")  # Replace "your_data.csv" with your actual file path

# Convert the last login column to datetime format
data['last_login'] = pd.to_datetime(data['Last login'])

# Extract the year from the last login column
data['login_year'] = data['last_login'].dt.year

# Count the number of users per year
user_counts = data['login_year'].value_counts().sort_index()

print(user_counts)

# Save the merged data to a new Excel file
user_counts.to_excel('users_by_year.xlsx', index=False)

login_year
2010.0       1
2011.0       1
2012.0       7
2013.0       5
2014.0       6
2015.0      16
2016.0      26
2017.0      17
2018.0      25
2019.0      23
2020.0     164
2021.0     353
2022.0     508
2023.0    1089
2024.0     882
Name: count, dtype: int64


**7. creating data for the "last login 180 days to 365 days ago" tab**

So here, we will use the output "merged_data" file to create the data for the "last login 180 days to 365 days ago" tab on this spreadsheet "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0".

This code does a few things:

1. It reads an Excel file called "merged_data.xlsx" into a DataFrame.
2. It removes duplicate rows based on the "Username" column.
3. It saves the cleaned data into a new Excel file called "result.xlsx".
4. It converts a column named "Last login" into datetime format.
5. It calculates today's date.
6. It calculates the date 180 days ago and the date 365 days ago from today's date.
7. It selects rows from the DataFrame where the "Last login" date falls between 180 and 365 days ago.
8. It saves the selected data into a new Excel file called "last_login_180_days_to_365_days_ago.xlsx".

Overall, it cleans and filters data related to user logins and saves the filtered data into a new Excel file.

Copy and paste results on this "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0"

In [29]:
# Read the Excel file into a DataFrame
excel_file = "/content/merged_data.xlsx"
df = pd.read_excel(excel_file)

# Remove duplicates based on the "Username" column
df.drop_duplicates(subset=["Username"], inplace=True)

# Save the merged data to a new Excel file
df.to_excel('result.xlsx', index=False)

In [30]:
# Assuming your DataFrame is named df and the column containing the last login dates is 'Last login'
# First, convert 'Last login' column to datetime if it's not already in datetime format
df['Date'] = pd.to_datetime(df['Last login'])

# Then, calculate today's date
today = pd.to_datetime('today')

# Calculate the date 180 days ago
start_date = today - pd.Timedelta(days=365)

# Calculate the date 365 days ago
end_date = today - pd.Timedelta(days=180)

# Select data within the specified range
selected_data = df[(df['Last login'] >= start_date) & (df['Last login'] <= end_date)]

# Now selected_data contains the rows where the last login was between 180 and 365 days ago


# Save the merged data to a new Excel file
selected_data.to_excel('last_login_180_days_to_365_days_ago.xlsx', index=False)

In [31]:
# to grab the user audit report, go to EMR website --> setup --> report --> user audit report --> run report and save to location desired

# Read the first Excel file containing the data
data_df = pd.read_excel('/content/User_Audit_Report_for_region_Colorado_03_29_2024.xlsx', skiprows=1)

# Read the other Excel file containing the "Username" data
username_df = pd.read_excel('last_login_180_days_to_365_days_ago.xlsx')

# Merge the filtered data with the "resource" data based on a common column (e.g., "resource")
merged_df = pd.merge(data_df, username_df, on='Username')

# Write the merged data to a new Excel file
merged_df.to_excel('last_login_180_days_to_365_days_ago_update_combined_user_audit_report.xlsx', index=False)

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**8. creating data for the "last login 180 days to 365 days ago" tab**

So here, we will use the output "merged_data" file to create the data for the "last login 180 days to 365 days ago" tab on this spreadsheet "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0".

This code does a few things:

1. It reads an Excel file called "merged_data.xlsx" into a DataFrame.
2. It removes duplicate rows based on the "Username" column.
3. It saves the cleaned data into a new Excel file called "result.xlsx".
4. It converts a column named "Last login" into datetime format.
5. It calculates today's date.
6. It calculates the date 180 days ago and the date 365 days ago from today's date.
7. It selects rows from the DataFrame where the "Last login" date falls between 180 and 365 days ago.
8. It saves the selected data into a new Excel file called "last_login_180_days_to_365_days_ago.xlsx".

Overall, it cleans and filters data related to user logins and saves the filtered data into a new Excel file.

Copy and paste results on this "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0"

In [ ]:
# Read the Excel file into a DataFrame
excel_file = "/content/merged_data.xlsx"
df = pd.read_excel(excel_file)

# Remove duplicates based on the "Username" column
df.drop_duplicates(subset=["Username"], inplace=True)

# Save the merged data to a new Excel file
df.to_excel('result.xlsx', index=False)

In [33]:
# Assuming your DataFrame is named df and the column containing the last login dates is 'Last login'
# First, convert 'Last login' column to datetime if it's not already in datetime format
df['Date'] = pd.to_datetime(df['Last login'])

# Then, calculate today's date
today = pd.to_datetime('today')

# Calculate the date threshold for last login (365 days ago)
date_threshold = today - pd.DateOffset(days=365)

# Select rows where last login is 365 days ago or more
filtered_df = df[df['Last login'] <= date_threshold]

# Save the merged data to a new Excel file
filtered_df.to_excel('last_login_365_days_ago_and_more.xlsx', index=False)


In [34]:
# to grab the user audit report, go to EMR website --> setup --> report --> user audit report --> run report and save to location desired

# Read the first Excel file containing the data
data_df = pd.read_excel('/content/User_Audit_Report_for_region_Colorado_03_29_2024.xlsx', skiprows=1)

# Read the other Excel file containing the "Username" data
username_df = pd.read_excel('/content/last_login_365_days_ago_and_more.xlsx')

# Merge the filtered data with the "resource" data based on a common column (e.g., "resource")
merged_df = pd.merge(data_df, username_df, on='Username')

# Write the merged data to a new Excel file
merged_df.to_excel('last_login_365_days_ago_and_more_update_combined_user_audit_report.xlsx', index=False)

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**9. creating data for the "all" tab**

So here, we will use the output "merged_data" file to create the data for the "all" tab on this spreadsheet "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0".

This code does the following:

1. It reads data from an Excel file named "merged_data.xlsx" and puts it into a DataFrame (think of it as a table in Python).
2. It removes any duplicate rows in the DataFrame based on the values in the "Username" column.
3. Finally, it saves the cleaned DataFrame into a new Excel file named "result.xlsx", without including the index column.

Copy and paste results on this "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0"

In [36]:
# Read the Excel file into a DataFrame
excel_file = "/content/merged_data.xlsx"
df = pd.read_excel(excel_file)

# Remove duplicates based on the "Username" column
df.drop_duplicates(subset=["Username"], inplace=True)

# Save the merged data to a new Excel file
df.to_excel('all.xlsx', index=False)

**10. creating data for the "never logged - last updated" tab**

So here, we will use the output "merged_data" file to create the data for the "never logged - last updated" tab on this spreadsheet "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0".

This code does this:

1. First, the code reads data from an Excel file called "merged_data.xlsx" and stores it in a DataFrame (a table-like structure used for data manipulation).

2. It then removes any duplicate rows in the DataFrame based on the values in the "Username" column. This ensures each username is unique in the dataset.

3. After removing duplicates, the cleaned data is saved to a new Excel file named "result.xlsx" without including the index column.

4. Next, the code reads the data from the "result.xlsx" file into a new DataFrame.

5. It checks for any blank values in the "Last login" column of the DataFrame.

6. Finally, it saves the rows with blank values in the "Last login" column to a new Excel file named "never_logged.xlsx".

In simpler terms, the code takes some data from an Excel file, cleans it up to remove duplicates, saves the cleaned-up version to a new file, then checks for any users who have never logged in and saves their information separately.

Copy and paste results on this "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0"

In [37]:
# Read the Excel file into a DataFrame
excel_file = "/content/merged_data.xlsx"
df = pd.read_excel(excel_file)

# Remove duplicates based on the "Username" column
df.drop_duplicates(subset=["Username"], inplace=True)

# Save the merged data to a new Excel file
df.to_excel('result.xlsx', index=False)

In [38]:
# Read the first Excel file containing the data
df = pd.read_excel('/content/result.xlsx')

# Check for blank values in the Last login column
blank_last_login = df[df['Last login'].isnull()]

# Save the merged data to a new Excel file
blank_last_login.to_excel('never_logged.xlsx', index=False)

**11. creating data for the "never logged - last updated merged with User Audit Report dataset " tab**

This is for the never logged update report merged with user audit report dataset. So this is only to update the "never logged - last updated" tab in this spreadsheet "https://docs.google.com/spreadsheets/d/1Qym5XWvc_o6zIj-kzoDGPc50KpmZa_KSc6pKtXTFgIQ/edit#gid=0". You are just updating the column "Last login" tab to see who logged in after emailing them.

This code does several things with Excel files using Python's pandas library:

1. It starts by reading an Excel file named "merged_data.xlsx" into a DataFrame (a table-like data structure).

2. It removes duplicate rows from the DataFrame based on the values in the "Username" column.

3. It saves the cleaned DataFrame to a new Excel file named "result.xlsx".

4. It checks for blank values in the "Last login" column of the DataFrame and creates a new DataFrame containing rows with blank "Last login" values.

5. It saves the DataFrame with blank "Last login" values to a new Excel file named "never_logged.xlsx".

6. It reads another Excel file named "User_Audit_Report_for_region_Colorado_(date of when it was downloaded).xlsx" **(data came from EMR --> setup --> user audit report --> download and upload to here)**, skipping the first row.

7. It merges the data from the two DataFrames based on the "Username" column.

8. It saves the merged DataFrame to a new Excel file named "never_logged_update_combined_user_audit_report.xlsx".


In [39]:
# Read the Excel file into a DataFrame
excel_file = "/content/merged_data.xlsx"
df = pd.read_excel(excel_file)

# Remove duplicates based on the "Username" column
df.drop_duplicates(subset=["Username"], inplace=True)

# Save the merged data to a new Excel file
df.to_excel('result.xlsx', index=False)

In [40]:
# Read the first Excel file containing the data
df = pd.read_excel('/content/result.xlsx')

# Check for blank values in the Last login column
blank_last_login = df[df['Last login'].isnull()]

# Save the merged data to a new Excel file
blank_last_login.to_excel('never_logged.xlsx', index=False)

In [41]:
# to grab the user audit report, go to EMR website --> setup --> report --> user audit report --> run report and save to location desired

# Read the first Excel file containing the data
data_df = pd.read_excel('/content/User_Audit_Report_for_region_Colorado_03_29_2024.xlsx', skiprows=1)

# Read the other Excel file containing the "Username" data
username_df = pd.read_excel('/content/never_logged.xlsx')

# Merge the filtered data with the "resource" data based on a common column (e.g., "resource")
merged_df = pd.merge(data_df, username_df, on='Username')

# Write the merged data to a new Excel file
merged_df.to_excel('never_logged_update_combined_user_audit_report.xlsx', index=False)


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**12. creating data for the "invalid users" tab based on email**

This code does the following:

1. It reads data from an Excel file called "merged_data.xlsx" into a DataFrame (a table-like structure for data).
2. It removes duplicate rows from the DataFrame based on the values in the "Username" and "Role" columns.
3. It saves the cleaned data to a new Excel file named "result.xlsx".
4. It reads the data from "result.xlsx" into a DataFrame.
5. It filters the DataFrame to select rows where the value in the "Status" column is 'Invalid'.
6. It saves the filtered data to a new Excel file named "invalid_df.xlsx".

So, in simple terms, it's cleaning up some data, removing duplicates, and then saving both the cleaned data and the rows marked as 'Invalid' to separate Excel files.

In [42]:
# Read the Excel file into a DataFrame
excel_file = "/content/merged_data.xlsx"
df = pd.read_excel(excel_file)

# Remove duplicates based on the "Username" column
df.drop_duplicates(subset=["Username"], inplace=True)

# Save the merged data to a new Excel file
df.to_excel('result.xlsx', index=False)

In [43]:
# Read the first Excel file containing the data
df = pd.read_excel('/content/result.xlsx')

# Assuming df is your DataFrame and "Status" is the column name
invalid_df = df[df['Status'] == 'Invalid']

# Write the merged data to a new Excel file
invalid_df.to_excel('invalid_df.xlsx', index=False)